In [ ]:
stats = {}

with open(input_file, 'r', encoding='utf-8') as f:
    reader = csv.DictReader(f) # -> [{...}, {...}, {...}]
    for row in reader:
        category = row['merchant_category']
        is_fraud = row['is_fraud'].strip().lower() == 'true'

        if category not in stats:
            stats[category] = {'total': 0, 'fraud': 0}

        stats[category]['total'] += 1
        if is_fraud:
            stats[category]['fraud'] += 1


# После обработки всех строк вы можете вывести результат на экран
# или сохранить в отдельный CSV.

output_file = 'merchant_category_stats.csv'


with open(output_file, 'w', newline='', encoding='utf-8') as out_f:
    fieldnames = ['merchant_category', 'total_transactions', 'fraud_transactions']
    writer = csv.DictWriter(out_f, fieldnames=fieldnames)
    writer.writeheader()
    for cat, counts in stats.items():
        writer.writerow({
        'merchant_category': cat,
        'total_transactions': counts['total'],
        'fraud_transactions': counts['fraud']
        })

print("Статистика успешно сохранена в", output_file)

In [ ]:
# Инструкция yield позволяет создавать генераторы.
# В отличие от объявления return в функции, где возвращается один объект, yield при каждом вызове функции
# генерирует новый объект.
# Фактически это дает возможность использовать генераторы в циклах.
# Самая важная причина применения такой инструкции - экономия памяти, когда не требуется сохранять всю
# последовательность, а можно получать ее элементы по одному. 

# Ученик написал генератор show_letters(some_str), выводящий все символы строки на печать, но только в
# том случае, если они являются буквами (остальные игнорируются).
# Сократите код функции.

def show_letters(some_str):
    clean_str = ''.join([letter for letter in some_str if letter.isalpha()])
    for symbol in clean_str:
        yield symbol
        
# Конструкция yield from позволяет полностью убрать цикл из функции. Она "вкладывает" один генератор
# внутрь другого, что дает возможность управления несколькими генераторами.

def show_letteres(some_str):
    yield from ''.join([letter for letter in some_str if letter.isalpha()])

random_str = show_letteres('A!sdf 09 _ w')

print(next(random_str))
print(next(random_str))

In [3]:
# Создать генератор, который будет делать то же самое что и range с одним аргументом.

print(list(range(5)))

def gen_range(end: int):
    num = 0
    while num < end:
        yield num
        num +=1
      

gen =  gen_range(6)       
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))
print(next(gen))


[0, 1, 2, 3, 4]
0
1
2
3
4
5


In [8]:
# Доработать предыдущий генератор так, чтобы он мог принимать один, два или три аргумента, как и обычный range.

def gen_range_1(*arg):
    if len(arg) == 1:
        end = arg[0]
        start = 0
        step = 1
    elif len(arg) == 2:
        start = arg[0]
        end = arg[1]
        step = 1
    elif len(arg) == 3:
        start = arg[0]
        end = arg[1]
        step = arg[2]
    else:
        raise TypeError(f"Вы передали {len(arg)} аргументов, ожидается от 1 до 3 параметров")
    if step == 0 or type(step) != int:
        raise ValueError("Шаг не должен равняться нулю и должен быть целым числом")
    
    current = start
    if step < 0:
        while current > end:
            yield current
            current += step
    else:
        while current < end:
            yield current
            current += step

gen_1 = gen_range_1(20, 2, -0.5)
print(next(gen_1))
print(next(gen_1))
print(next(gen_1))
print(next(gen_1))
print(next(gen_1))
print(next(gen_1))

ValueError: Шаг не должен равняться нулю и должен быть целым числом

In [9]:
# Пример использования:


In [ ]:
# Написать функцию validateCustomers(customer), которая принимает на входе список кортежей:
# Имя
# Фамилия
# Дата рождения
# Номер банковского счета (iban)
# 
# Функция возвращает мапу, где ключом является строка имя+фамилия, а значением - список сообщений об ошибках, возникших при валидации пользователя по следующим правилам:
# Имя и фамилия не должны быть пустыми.
# Возраст валиден, если он больше 18 лет. 
# Iban должен соответствовать стандарту длины (начинаться с кода страны и содержать правильное количество символов)
# 
# Решение должно использовать исключения и итераторы. 
from _datetime import datetime
customers_data = [
("Иван", "Иванов", "2005-05-10", "DE1234567890123456789012"), # Молод, будет ошибка возраста
("", "Петров", "1990-01-01", "DE1234567890123456789012"), # Пустое имя
("Сидор", "Сидоров", "1980-12-12", "DE123456789012345"), # Неправильный IBAN (короткий)
("Анна", "Каренина", "1970-07-07", "DE1234567890123456789012") # Всё ок
]

import string

def validate_fullname(name, surname):
    if not name.strip():
        raise ValueError("Пустое поле ИМЯ!")
    elif not surname.strip():
        raise ValueError("Пустое поле ФАМИЛИЯ!")
        
        
def validate_age(birthday):
    try:
        birthday = datetime.strptime(birthday, '%Y-%m-%d')
    except ValueError as err:
        return "Неверный формат даты"
    age = datetime.today().year - birthday.year
    if (datetime.today().month, datetime.today().day) < (birthday.month, birthday.day):
        age -= 1
    if age < 18:
        raise ValueError("Возраст должен быть больше 18 лет")
    
    
def validate_iban(iban: str):
    if not iban.upper().startswith("DE"):
        raise ValueError("IBAN принадлежит другому региону. IBAN должен начинаться с DE")
    if len(iban) != 22:
        raise ValueError("IBAN не корректный. Должен быть равен 22 символам")
        
             
def validateCustomers(customers):
    for customer in customers:
        validate_fullname(customer[0], customer[1])
        validate_age(customer[2])
        validate_iban(customer[3])
        yield customer
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    